In [64]:
import json
import pandas as pd

"""
Method
"""

# Load cleaned song meta data(79,814)
def load_song_meta() -> pd.DataFrame:
    with open('./cleaned_song_meta.json', 'r', encoding='UTF8') as f:
        data = json.load(f)
    df = pd.json_normalize(data)
    df = df[['id', 'song_name', 'artist_name_basket']]
    df['tags'] = [[] for _ in range(len(df))]
    df['count'] = 0
    return df

# Load playlist data(115,071)
def load_playlist() -> pd.DataFrame:
    with open('./train.json', 'r', encoding='UTF8') as f:
        data = json.load(f)
    df = pd.json_normalize(data)
    df.sort_values(by=['like_cnt', 'updt_date'], ascending=False, inplace=True)
    return df

# Add playlist's @tags to each songs and update @count with weight.
def processing_playlist(song_df: pd.DataFrame, playlist_df: pd.DataFrame) -> None:
    for idx_, row in playlist_df.iterrows():
        cnt = row.like_cnt
        song_list = row.songs
        tags = row.tags
        for song in song_list:
            if song in song_df['id'].values:
                idx, = song_df.index[song_df['id'] == song].values
                # use weight for cnt / 1000 (e.g., 53211 -> 53, 3523 -> 3) (pending)
                song_df.at[idx, 'count'] += int(cnt/1000)
                origin_tags = dict(song_df.at[idx, 'tags'])
                
                for tag in tags:
                    if tag not in origin_tags: origin_tags[tag] = 1
                    elif tag in origin_tags: origin_tags[tag] += 1
                
                if len(origin_tags) > 20:
                    new_dict = {}
                    for tag, cnt in sorted(list(origin_tags.items()), key=lambda x: -x[1])[:20]:
                        new_dict[tag] = cnt
                    song_df.at[idx, 'tags'] = list(new_dict.items())
                else:
                    song_df.at[idx, 'tags'] = list(origin_tags.items())

In [65]:
song = load_song_meta()
playlist = load_playlist()
processing_playlist(song, playlist)

In [68]:
song.sort_values(by='count', ascending=False, inplace=True)
song.head(10)

,id,song_name,artist_name_basket,tags,count
41312,366786,가끔 미치도록 네가 안고 싶어질 때가 있어,[가을방학],"[(새벽, 373), (인디, 321), (밤, 317), (감성, 294), (발...",118
38233,339802,홀로 (Feat. 김나영),[정키],"[(발라드, 350), (새벽, 233), (추억, 214), (밤, 198), (...",88
14889,133143,"그대와 나, 설레임 (Feat. 소울맨)",[어쿠스틱 콜라보],"[(기분전환, 276), (힐링, 208), (카페, 143), (발라드, 140)...",87
69494,616212,너를 그리다,[디어 (d.ear)],"[(사랑, 160), (카페, 110), (설렘, 106), (기분전환, 93), ...",86
55590,494024,첫째 날 (With 나얼),[Ecobridge],"[(발라드, 121), (새벽, 113), (슬픔, 109), (밤, 97), (감...",80
78942,700348,"Walking in the Moonlight (Feat. 다원, Lazier)",[서교동의 밤],"[(감성, 142), (새벽, 84), (잔잔한, 74), (인디, 73), (휴식...",78
70562,625996,Autumn Breeze (Feat. Rachel Lim),[JIDA (지다)],"[(감성, 143), (새벽, 84), (잔잔한, 72), (밤, 58), (인디,...",71
49897,443902,밤하늘의 별을... (With KCM & No Noo),[양정승],"[(사랑, 115), (발라드, 109), (추억, 85), (기분전환, 76), ...",66
68898,610933,널 생각해,[원 모어 찬스 (one more chance)],"[(사랑, 539), (설렘, 398), (기분전환, 264), (힐링, 217),...",65
76181,675115,야생화,[박효신],"[(발라드, 476), (감성, 280), (이별, 249), (추억, 220), ...",65
